# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2c21c0b7-d449-4766-8363-58a27fe698a9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 173ms :: artifacts dl 4m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 2

Daily gainers

In [5]:
task2_daily = (
    parsed_df
        .withWatermark("timestamp", "10 minutes")
        .groupBy(window("timestamp", "5 minutes"), "name")
        .agg(
            first("price").alias("price_before"),
            last("price").alias("price_after"),
            (last("price") - first("price")).alias("daily_gain"),
            ((last("price") - first("price")) / first("price") * 100).alias("percentage_gain"),
            count("*").alias("n_events")
        )
        .orderBy(desc("daily_gain"))
)

task2_daily_query = (
    task2_daily.writeStream
        .format("console")
        .outputMode("complete")
        .option("truncate", False)
        .start()
)

25/11/24 20:58:42 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-442d7ccb-c7cc-4858-bfef-4b21a4439f55. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+----+------------+-----------+----------+---------------+--------+
|window                                    |name|price_before|price_after|daily_gain|percentage_gain|n_events|
+------------------------------------------+----+------------+-----------+----------+---------------+--------+
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|BA  |133.08      |133.08     |0.0       |0.0            |1       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|ALGN|96.5681     |96.5681    |0.0       |0.0            |1       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|EW  |117.46      |117.46     |0.0       |0.0            |1       |
+------------------------------------------+----+------------+-----------+----------+---------------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+------------+-----------+----------+------------------+--------+
|window                                    |name|price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+----+------------+-----------+----------+------------------+--------+
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|MLM |183.79      |227.55     |43.76001  |23.80978908764493 |4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|GD  |152.18      |175.31     |23.130005 |15.19911026155118 |2       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|FFIV|119.44      |141.895    |22.455002 |18.800235576075487|4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PH  |123.18      |145.52     |22.340004 |18.1360642246617  |3       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|SNA |149.98      |166.67     |16.690002 |11.128152364883459|2

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PCLN |1546.67     |1772.49    |225.81995 |14.600395680582995|2       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|MLM  |183.79      |227.55     |43.76001  |23.80978908764493 |4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|BRK.B|144.76      |175.85     |31.090012 |21.47693615393964 |4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|GD   |152.18      |175.31     |23.130005 |15.19911026155118 |2       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|FFIV |119.44      |141.895    |22.455002 |18.800235576

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PCLN |1546.67     |1798.75    |252.07996 |16.298237432185026|3       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|MLM  |183.79      |227.55     |43.76001  |23.80978908764493 |4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|GD   |152.18      |194.07     |41.890015 |27.526624171735886|4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|MHK  |199.8       |240.2      |40.399994 |20.220216856562725|4       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|INCY |105.02      |141.24     |36.22001  |34.488678354

...

-------------------------------------------
Batch: 134
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOG |794.23      |1049.7     |255.46997 |32.165742541265956|31      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|CMG  |368.39      |535.64     |167.25    |45.40025335909559 |55      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOGL|800.17      |952.8      |152.63    |19.074697644581594|54      |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|REGN |285.55      |394.645    |109.095   |38.20522006108432 |3       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PRGO |86.93       |195.0      |108.07    |124.318416

-------------------------------------------
Batch: 135
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOG |794.23      |1049.7     |255.46997 |32.165742541265956|31      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|CMG  |368.39      |535.64     |167.25    |45.40025335909559 |55      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOGL|800.17      |952.8      |152.63    |19.074697644581594|54      |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|REGN |285.55      |394.645    |109.095   |38.20522006108432 |3       |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PRGO |86.93       |195.0      |108.07    |124.318416

-------------------------------------------
Batch: 136
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOG |794.23      |1049.7     |255.46997 |32.165742541265956|31      |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|REGN |285.55      |475.95     |190.40002 |66.67835144580974 |4       |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|CMG  |368.39      |535.64     |167.25    |45.40025335909559 |55      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOGL|800.17      |952.8      |152.63    |19.074697644581594|54      |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PRGO |86.93       |195.0      |108.07    |124.318416

-------------------------------------------
Batch: 137
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOG |794.23      |1049.7     |255.46997 |32.165742541265956|31      |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|REGN |285.55      |514.91     |229.35999 |80.32218356032797 |6       |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|CMG  |368.39      |535.64     |167.25    |45.40025335909559 |55      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOGL|800.17      |952.8      |152.63    |19.074697644581594|54      |
|[2025-11-24 20:55:00, 2025-11-24 21:00:00]|PRGO |86.93       |195.0      |108.07    |124.318416

-------------------------------------------
Batch: 138
-------------------------------------------
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|window                                    |name |price_before|price_after|daily_gain|percentage_gain   |n_events|
+------------------------------------------+-----+------------+-----------+----------+------------------+--------+
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOG |794.23      |1049.7     |255.46997 |32.165742541265956|31      |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|REGN |285.55      |514.91     |229.35999 |80.32218356032797 |6       |
|[2025-11-24 21:05:00, 2025-11-24 21:10:00]|AZO  |545.649     |747.25     |201.60101 |36.94701503247291 |4       |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|CMG  |368.39      |535.64     |167.25    |45.40025335909559 |55      |
|[2025-11-24 21:00:00, 2025-11-24 21:05:00]|GOOGL|800.17      |952.8      |152.63    |19.0746976